In [ ]:
cd drive/My\ Drive/app/EGR590

/content/drive/My Drive/app/EGR590


In [ ]:
import numpy as np
from pathlib import Path
import re

0 - original,
1 - random,
2 - close,
3 - far

In [ ]:
preproced_docs =[]
for file in Path("./preproc").rglob("*.txt"):
    with open(file) as f:
        txt_file_as_string = f.read()
    preproced_docs.append(txt_file_as_string)

base_document = preproced_docs[0]
documents = preproced_docs[1:]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

reference = base_document
comparison_docs = documents

def process_tfidf_similarity():
  vectorizer = TfidfVectorizer()

	# To make uniformed vectors, both documents need to be combined first.
  embeddings = vectorizer.fit_transform(preproced_docs)
  
  cosine_similarities = cosine_similarity(embeddings[0:1], embeddings[0:]).flatten()
  
  highest_score = 0
  highest_score_index = 0
  
  for i, score in enumerate(cosine_similarities):
    print(i, score)
    if highest_score < score:
      highest_score = score
      highest_score_index = i
  
  most_similar_document_rem = documents[highest_score_index]
  
  print("Most similar document by TF-IDF with the score:", highest_score_index, highest_score)

process_tfidf_similarity()

0 1.0
1 0.8829193112698767
2 0.9015492352341855
3 0.7004992765627059
Most similar document by TF-IDF with the score: 0 1.0


In [ ]:
#!wget "https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed" universal-sentence-encoder_4

In [ ]:
#!tar -xvf 4?tf-hub-format=compressed

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
import tensorflow_hub as hub

def process_use_similarity():
  filename = "./USEmodel"
  model = hub.load(filename)
  
  base_embeddings = model([base_document])
  
  embeddings = model(preproced_docs)
  
  scores = cosine_similarity(base_embeddings, embeddings).flatten()
  
  highest_score = 0
  highest_score_index = 0
  for i, score in enumerate(scores):
    print(i, score)
    if highest_score < score:
      highest_score = score
      highest_score_index = i
      
  most_similar_document = documents[highest_score_index]
  print("Most similar document by USE with the score:", highest_score_index, highest_score)

process_use_similarity()

0 1.0
1 0.44078398
2 0.5280792
3 0.43918502
Most similar document by USE with the score: 0 1.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from nltk import sent_tokenize

from sentence_transformers import SentenceTransformer

def process_bert_similarity():
  model = SentenceTransformer('bert-base-nli-mean-tokens')

  sentences = sent_tokenize(base_document)
  base_embeddings_sentences = model.encode(sentences)
  base_embeddings = np.mean(np.array(base_embeddings_sentences), axis=0)
  
  vectors = []
  
  for i, document in enumerate(documents):
    sentences = sent_tokenize(document)
    embeddings_sentences = model.encode(sentences)
    embeddings = np.mean(np.array(embeddings_sentences), axis=0)
    
    vectors.append(embeddings)
    print("making vector at index:", i)
    
  vectors.insert(0, base_embeddings)
  scores = cosine_similarity([base_embeddings], vectors).flatten()
    
  highest_score = 0
  highest_score_index = 0
  for i, score in enumerate(scores):
    print(i, score)
    if highest_score < score:
      highest_score = score
      highest_score_index = i
  
  most_similar_document = documents[highest_score_index]
  print("Most similar document by BERT with the score:", highest_score_index, highest_score)

process_bert_similarity()

making vector at index: 0
making vector at index: 1
making vector at index: 2
0 1.0000002
1 0.827144
2 0.90516925
3 0.6855231
Most similar document by BERT with the score: 0 1.0000002


In [ ]:
#!wget "https://ai2-s2-research.s3-us-west-2.amazonaws.com/longformer/longformer-base-4096.tar.gz"

In [ ]:
#!tar -xvf longformermodel/longformer-base-4096.tar.gz

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import LongformerModel, LongformerTokenizer, LongformerConfig

def process_longformer_similarity():

  model = LongformerModel.from_pretrained('./longformermodel/longformer-base-4096')
  tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

  base_tokenized = torch.tensor(tokenizer.encode(base_document, truncation=True)).unsqueeze(0)  # batch of size 1
  base_embedding = model(base_tokenized)
  print(base_embedding)
  exit(1)
  base_embedding = torch.mean(torch.stack(base_embedding, 0))

  scores = cosine_similarity(base_embedding, base_embedding).flatten()
  print(scores)

process_longformer_similarity()

Some weights of the model checkpoint at ./longformermodel/longformer-base-4096 were not used when initializing LongformerModel: ['roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.self.query_global.weight', 'roberta.encoder.layer.0.attention.self.query_global.bias', 'roberta.encoder.layer.0.attention.self.key_global.weight', 'roberta.encoder.layer.0.attention.self.key_global.bias', 'roberta.encoder.layer.0.attention.self.value_global.weight', 'roberta.encoder.layer.0.att

(tensor([[[ 0.1382, -0.6664, -1.4531,  ..., -1.7736, -1.5396,  0.7059],
         [ 0.8418,  0.7191, -1.9285,  ..., -2.8762, -0.0929, -0.3925],
         [ 2.7008,  0.6449, -1.9766,  ..., -0.3545, -0.7933,  0.4709],
         ...,
         [ 1.4051,  0.1597, -0.5038,  ..., -1.3898, -1.0105, -0.4175],
         [ 2.4175, -0.0375, -0.9381,  ..., -1.0539,  0.1333, -0.2646],
         [ 1.6907,  0.4530, -0.2882,  ..., -1.2306, -0.1024, -0.3290]]],
       grad_fn=<NativeLayerNormBackward>), tensor([[-5.0158e-01,  6.9932e-02,  9.2705e-01,  4.6227e-01, -3.7044e-01,
         -3.8549e-01,  7.4490e-01, -8.7140e-01, -3.8253e-01,  2.9432e-01,
         -3.5054e-01,  1.9231e-01, -1.2834e-01,  1.8306e-01,  2.5327e-01,
          3.4146e-01,  4.5417e-01,  6.7930e-01,  4.2687e-01, -8.1116e-01,
         -3.6495e-01,  3.6333e-02, -5.7067e-01,  6.0623e-01,  1.4249e-01,
          1.8065e-01,  5.0042e-03,  2.0572e-01,  2.7808e-01,  8.8426e-02,
          8.7566e-01,  8.5729e-01,  2.8682e-01, -3.3233e-01, -1.6347e-

RuntimeError: ignored